In [1]:
import pandas as pd
import numpy as np
import re
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import cv2

In [2]:
img = cv2.imread('sample_invoice.PNG')
img2 = cv2.resize(img ,None,fx=1.5,fy=1.5)
gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
adaptive = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,87,11)

In [4]:
cv2.namedWindow('Image',cv2.WINDOW_AUTOSIZE)
cv2.imshow('Image',adaptive)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [7]:
text = pytesseract.image_to_string(adaptive)
print(text)

   
   
 
 

yo.
RAGA PHARMACY
0:75, KANOY ROAD, KADAWATHA,
7 TEL n O12 92196

« Units 3 Date/Time:27/01/2021 07:06:46 Pt
Cashier: KAUSHALYA Receipt No:00000139N
SH 5 LEDSS ~ ESHARA

 

 

a MEGAMAK FISH OIL
N43 MAS (60. O00 & 27, oe 40
SB TW os TBM

— oQ 2 ewonsns 7 4
TRIER MBER A

(C) Biz Net Teclmalogies - eras :

